## Biblioteki

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

## Rodzaj ruchów

In [6]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Koszt

In [7]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Pobór danych

In [8]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Algorytm IHC

In [11]:
# Mapowanie typów sąsiedztwa na funkcje
neighborhood_functions = {
    "swap": swap_move,
    "two_opt": two_opt_move,
    "insertion": insertion_move
}

def hill_climbing(distance_matrix, initial_route, neighborhood_type, max_iterations_without_improvement, max_hill_climbing_iterations):
    current_route = initial_route
    current_distance = calculate_path_cost(distance_matrix, current_route)
    
    # Wybierz funkcję sąsiedztwa na podstawie typu
    neighbor_func = neighborhood_functions[neighborhood_type]
    iterations_without_improvement = 0
    total_iterations = 0

    while iterations_without_improvement < max_iterations_without_improvement and total_iterations < max_hill_climbing_iterations:
        improved = False
        # Przeglądaj sąsiedztwa
        for neighbor in neighbor_func(current_route):
            new_distance = calculate_path_cost(distance_matrix, neighbor)
            if new_distance < current_distance:
                current_route = neighbor
                current_distance = new_distance
                improved = True
                iterations_without_improvement = 0
                break  # Skok do następnego sąsiedztwa

        if not improved:
            # Jeśli nie znaleziono lepszego rozwiązania, zwiększ licznik iteracji bez poprawy
            iterations_without_improvement += 1
        
        # Zwiększ licznik iteracji
        total_iterations += 1

    return current_route, current_distance

def iterated_hill_climbing(distance_matrix, num_starts=100, neighborhood_type="swap", time_limit=1000, max_iterations_without_improvement=100, max_hill_climbing_iterations=1000):
    start_time = time.time()  # Czas rozpoczęcia

    n = len(distance_matrix)
    best_route = None
    best_distance = float('inf')
    best_time = float('inf')

    for start in range(num_starts):
        # Sprawdzenie limitu czasu
        if time_limit and (time.time() - start_time > time_limit):
            print(f"Przekroczono limit czasu ({time_limit} s).")
            break

        # Losowe rozwiązanie początkowe
        initial_route = list(range(n))
        random.shuffle(initial_route)

        # Wspinaczka
        hill_climbing_start_time = time.time()
        current_route, current_distance = hill_climbing(distance_matrix, initial_route, neighborhood_type, max_iterations_without_improvement, max_hill_climbing_iterations)
        hill_climbing_time = time.time() - hill_climbing_start_time

        # Aktualizacja najlepszego rozwiązania
        if current_distance < best_distance:
            best_route = current_route
            best_distance = current_distance
            best_time = hill_climbing_time

    best_solution = [x + 1 for x in best_solution]  # Zmień indeksy na numery miast
    return best_route, best_distance, best_time

## Generowanie wyników

Za podstawowe dane przyjmujemy:

* num_starts = 100, 

* neighborhood_type = "swap", 

* time_limit = 1000, 

* max_iterations_without_improvement = 100, 

* max_hill_climbing_iterations = 1000.

In [ ]:
# Dane_TSP_48
best_path_1, best_cost_1, best_time_1 = iterated_hill_climbing(data1)
# Dane_TSP_76
best_path_2, best_cost_2, best_time_2 = iterated_hill_climbing(data2)
# Dane_TSP_127
best_path_3, best_cost_3, best_time_3 = iterated_hill_climbing(data3)

In [ ]:
W1 =  {
    "Długość ścieżki": [best_cost_1, best_cost_2, best_cost_3],
    "Ścieżka": [best_path_1, best_path_2, best_path_3],
    "Czas": [best_time_1, best_time_2, best_time_3]
}

basic = pd.DataFrame(data = W1)

with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='w') as writer:
    basic.to_excel(writer, sheet_name = "Basic", index=False)

Badanie wpływu parametru *num_starts*

In [ ]:
def test_num_starts(data, dataset_name, num_repeats=10):
    results = []
    num_starts_values = [50, 75, 100, 125, 150]
    for num_starts in num_starts_values:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, num_starts=num_starts)
            results.append({"PARAMETR": num_starts, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_num_starts_1 = test_num_starts(data1, "DATA1")
df_num_starts_2 = test_num_starts(data2, "DATA2")
df_num_starts_3 = test_num_starts(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_num_starts_1, df_num_starts_2, df_num_starts_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "num_starts", index=False)

Badanie wpływu parametru *neighborhood_type*

In [ ]:
def test_neighborhood_type(data, dataset_name, num_repeats=10):
    results = []
    neighborhood_types = ["swap", "two_opt", "insertion"]
    for neighborhood_type in neighborhood_types:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, neighborhood_type=neighborhood_type)
            results.append({"PARAMETR": neighborhood_type, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_neighborhood_type_1 = test_neighborhood_type(data1, "DATA1")
df_neighborhood_type_2 = test_neighborhood_type(data2, "DATA2")
df_neighborhood_type_3 = test_neighborhood_type(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
neighborhood_type_results = pd.concat([df_neighborhood_type_1, df_neighborhood_type_2, df_neighborhood_type_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    neighborhood_type_results.to_excel(writer, sheet_name="neighborhood_type", index=False)

Badanie wpływu parametru *time_limit*

In [ ]:
def test_time_limit(data, dataset_name, num_repeats=10):
    results = []
    time_limits = [500, 1000, 1500, 2000, 2500]
    for time_limit in time_limits:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, time_limit=time_limit)
            results.append({"PARAMETR": time_limit, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_time_limit_1 = test_time_limit(data1, "DATA1")
df_time_limit_2 = test_time_limit(data2, "DATA2")
df_time_limit_3 = test_time_limit(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
time_limit_results = pd.concat([df_time_limit_1, df_time_limit_2, df_time_limit_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    time_limit_results.to_excel(writer, sheet_name="time_limit", index=False)

Badanie wpływu parametru *max_iterations_without_improvement*

In [ ]:
def test_max_iterations_without_improvement(data, dataset_name, num_repeats=10):
    results = []
    max_iterations_values = [50, 75, 100, 125, 150]
    for max_iterations in max_iterations_values:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, max_iterations_without_improvement=max_iterations)
            results.append({"PARAMETR": max_iterations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_max_iterations_1 = test_max_iterations_without_improvement(data1, "DATA1")
df_max_iterations_2 = test_max_iterations_without_improvement(data2, "DATA2")
df_max_iterations_3 = test_max_iterations_without_improvement(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
max_iterations_results = pd.concat([df_max_iterations_1, df_max_iterations_2, df_max_iterations_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    max_iterations_results.to_excel(writer, sheet_name="max_iterations_without_improvement", index=False)

Badanie wpływu parametru *max_hill_climbing_iterations*

In [ ]:
def test_max_hill_climbing_iterations(data, dataset_name, num_repeats=10):
    results = []
    max_hill_climbing_iterations_values = [500, 750, 1000, 1250, 1500]
    for max_iterations in max_hill_climbing_iterations_values:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, max_hill_climbing_iterations=max_iterations)
            results.append({"PARAMETR": max_iterations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_max_hill_climbing_1 = test_max_hill_climbing_iterations(data1, "DATA1")
df_max_hill_climbing_2 = test_max_hill_climbing_iterations(data2, "DATA2")
df_max_hill_climbing_3 = test_max_hill_climbing_iterations(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
max_hill_climbing_results = pd.concat([df_max_hill_climbing_1, df_max_hill_climbing_2, df_max_hill_climbing_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    max_hill_climbing_results.to_excel(writer, sheet_name="max_hill_climbing_iterations", index=False)